In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind"
label_col = "label"

# 포함할 abnormal return 컬럼
abn_return_cols = [
    "abn_ret_minus_10m", "abn_ret_minus_9m", "abn_ret_minus_8m", "abn_ret_minus_7m", "abn_ret_minus_6m",
    "abn_ret_minus_5m", "abn_ret_minus_4m", "abn_ret_minus_3m", "abn_ret_minus_2m", "abn_ret_minus_1m",
    "abn_ret_1m", "abn_ret_2m", "abn_ret_3m", "abn_ret_4m", "abn_ret_5m",
    "abn_ret_6m", "abn_ret_7m", "abn_ret_8m", "abn_ret_9m", "abn_ret_10m"
]

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

# 2021-01-01 ~ 2022-06-30(포함) event_ts 조건 추가
query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
WHERE ar.event_ts >= '2021-01-01' AND ar.event_ts <= '2022-06-30'
ORDER BY ar.event_ts ASC
"""

df_before_telegram = pd.read_sql(query, engine)
df_before_telegram["period_dummy"] = 0
df_before_telegram

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,abn_ret_3m,abn_ret_4m,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,period_dummy
0,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,-0.18,0.19,-0.20,-0.23,-0.24,-0.67,-0.34,-0.45,0,0
1,2021-01-04 10:08:00+00:00,NaN,NaN,-1.45,-1.64,-1.44,-2.24,-2.49,-2.69,-1.96,...,-2.66,-2.49,-2.27,-1.24,-1.62,-1.37,-1.04,-1.91,1,0
2,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,-0.18,0.19,-0.20,-0.23,-0.24,-0.67,-0.34,-0.45,0,0
3,2021-01-04 10:14:00+00:00,-8.40,-8.59,-8.32,-8.32,-8.19,-8.65,-8.42,-8.36,-8.10,...,-2.60,-4.00,-3.64,-3.51,-3.64,-4.17,-4.70,-4.90,1,0
4,2021-01-04 10:16:00+00:00,0.22,0.50,0.23,0.21,0.23,0.23,0.15,0.01,-0.31,...,0.25,0.19,0.06,-0.24,-0.06,-0.24,-0.50,0.03,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,2022-06-29 14:04:00+00:00,2.30,1.83,1.79,2.27,2.25,0.06,2.27,1.87,1.83,...,3.51,1.29,2.22,1.34,1.41,1.45,1.40,0.55,1,0
5068,2022-06-29 14:20:00+00:00,-0.16,-0.08,-0.05,-0.10,-0.05,-0.03,-0.10,-0.16,-0.18,...,0.03,-0.16,-0.16,0.03,-0.10,-0.03,0.00,-0.03,1,0
5069,2022-06-29 14:21:00+00:00,0.00,0.15,-0.02,-0.09,-0.07,-0.02,-0.08,0.03,-0.08,...,0.04,-0.08,-0.06,0.11,0.29,0.20,0.17,0.06,1,0
5070,2022-06-29 14:35:00+00:00,0.14,0.16,0.06,0.13,-0.13,-0.02,0.00,-0.29,-0.06,...,0.08,0.27,0.29,0.02,-0.02,-0.05,-0.06,-0.22,1,0


In [3]:
label_table = "label"
return_table = "abnormal_return"
label_col = "label"


all_sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"] +
    ["k.report_name"] +
    ["k.summary_kr"]
)
all_sql_columns_str = ",\n    ".join(all_sql_columns)

query = f"""
SELECT {all_sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
JOIN disclosure_events de ON ar.event_id = de.id
WHERE k.{label_col} IS NOT NULL
ORDER BY ar.event_ts ASC
"""

df_after_telegram = pd.read_sql(query, engine)
df_after_telegram["period_dummy"] = 1
label_map = {0: 1, 1: 0, 2: -1}
df_after_telegram['label'] = df_after_telegram['label'].map(label_map)
df_after_telegram

,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,report_name,summary_kr,period_dummy
0,2022-07-01 01:03:17+00:00,-1.80,-0.66,0.14,0.19,-0.19,-0.19,0.28,-0.14,-0.14,...,0.00,-0.28,-0.66,-0.33,-0.80,-1.27,1,타법인주식및출자증권취득결정(자율공시),"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...",1
1,2022-07-01 01:24:45+00:00,-1.57,-1.43,-1.43,-1.43,-1.28,-1.23,-1.19,-1.66,-1.66,...,0.86,0.48,0.10,0.48,0.10,0.53,1,단일판매ㆍ공급계약체결,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,1
2,2022-07-01 01:27:02+00:00,-0.05,0.10,0.00,0.05,-0.14,-0.14,-0.14,-0.05,0.14,...,-0.14,-0.24,-0.24,-0.14,0.14,0.29,1,단일판매ㆍ공급계약체결(자회사의 주요경영사항),비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,1
3,2022-07-01 01:43:15+00:00,-0.89,-0.99,-1.11,-1.13,-1.03,-1.13,-1.18,-1.26,-1.34,...,-0.20,0.74,0.09,0.75,-0.04,0.08,1,신규시설투자등(자율공시),동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,1
4,2022-07-01 02:30:09+00:00,-1.91,-2.23,-1.81,-2.09,-1.91,-1.86,-1.91,-1.96,-1.78,...,-0.80,-0.40,-0.67,-0.22,-0.40,-0.40,-1,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,2023-12-28 05:53:08+00:00,-0.08,0.64,0.68,0.62,-0.01,0.02,0.00,0.00,-0.01,...,0.21,0.21,0.23,0.27,0.27,0.24,1,단일판매ㆍ공급계약체결,까뮤이앤씨는 조달청과 전라남도 목포시에서 수산식품 수출단지 조성사업 건축공사(공급지...,1
8759,2023-12-28 05:55:03+00:00,-0.31,-0.26,-0.02,-0.10,-0.38,-0.53,-0.34,-0.43,-0.55,...,-2.83,-2.15,-1.84,-1.08,-1.45,-2.10,0,조회공시요구(현저한시황변동)에대한답변(미확정),"에코프로머티리얼즈는 신규 고객 확보와 관련된 사업 협의를 진행 중이며, 아직 확정된...",1
8760,2023-12-28 05:56:27+00:00,0.05,-0.08,0.11,0.17,0.00,-0.17,-0.04,-0.26,-0.19,...,-0.25,-0.45,-0.22,-0.41,-0.43,-0.28,1,[기재정정]연결재무제표기준영업실적등에대한전망(공정공시),"에코마케팅은 2023년 매출액 3,500억 원과 영업이익 530억 원을 기록할 것으...",1
8761,2023-12-28 05:56:32+00:00,0.05,-0.08,0.11,0.17,0.00,-0.17,-0.04,-0.26,-0.19,...,-0.25,-0.45,-0.22,-0.41,-0.43,-0.28,1,[기재정정]영업실적등에대한전망(공정공시),"에코마케팅은 2023년 매출액이 490억 원, 영업이익이 200억 원으로 예상되며,...",1


In [4]:
from config import keywords
import re
import os

def classify_report_name_with_keywords_return_series_and_list(df, keywords):
    """
    df: DataFrame, must include 'report_name' column
    keywords: dict, key=disclosure_type, value=[regex strings]
    
    Returns: (Series of classified disclosure_type, List of disclosure_types in order of df)
    """
    pattern_list = []
    for disclosure_type, pattern_strs in keywords.items():
        if pattern_strs:
            for pat in pattern_strs:
                pattern_list.append( (disclosure_type, re.compile(pat)) )
        else:
            pattern_list.append( (disclosure_type, re.compile(disclosure_type)) )

    def classify_single_report(report_name):
        for disclosure_type, pat in pattern_list:
            if isinstance(report_name, str):
                if pat.search(report_name):
                    return disclosure_type
        return None

    result_list = df["report_name"].apply(classify_single_report).tolist()
    return pd.Series(result_list, index=df.index), result_list

output_dir = "type"
os.makedirs(output_dir, exist_ok=True)

# 분류도 csv로 저장하고 df_after_telegram['disclosure_type']에도 넣기
disclosure_type_series, disclosure_type_list = classify_report_name_with_keywords_return_series_and_list(
    df_after_telegram, keywords)
df_after_telegram["disclosure_type"] = disclosure_type_series

# 각 event type(disclosure_type) 별로 따로 csv 저장 (type 폴더에!)
for dtype, group in df_after_telegram.groupby("disclosure_type"):
    if dtype is None:
        continue  # 기타 처리로 분리
    fname = f"{dtype}.csv".replace("/", "_").replace("\\", "_")
    path = os.path.join(output_dir, fname)
    # summary_kr도 같이 저장 (없으면 없는 컬럼으로 저장)
    cols_to_save = ["report_name"]
    if "summary_kr" in group.columns:
        cols_to_save.append("summary_kr")
    group[cols_to_save].to_csv(path, index=False)

# 분류되지 않은 event는 "기타.csv"로 저장
not_classified = df_after_telegram[df_after_telegram['disclosure_type'].isna()]
if not not_classified.empty:
    etc_path = os.path.join(output_dir, "기타.csv")
    cols_to_save = ["report_name"]
    if "summary_kr" in not_classified.columns:
        cols_to_save.append("summary_kr")
    not_classified[cols_to_save].to_csv(etc_path, index=False)


In [5]:
from config import keywords

# 제외할 disclosure_type 목록
exclude_types = ["실적공시", "지분공시", "IR활동", "배당"]

# disclosure_type이 keywords key에 있으면서 exclude_types에는 없는 값만 보여주기
filtered_types = [k for k in keywords.keys() if k not in exclude_types]
df_after_telegram[df_after_telegram["disclosure_type"].isin(filtered_types)]["disclosure_type"].value_counts()

disclosure_type
단일판매ㆍ공급계약체결           1054
매출액변동                  351
자기주식취득 신탁계약 체결 결정      205
타법인 주식 및 출자증권 양수결정     110
자기주식 취득 결정              75
특허권취득                   74
자기주식 처분 결정              68
신규시설투자                  67
유상증자 결정                 62
임상 계획 신청                51
무상증자 결정                 49
소송등의판결ㆍ결정               28
임상 계획 승인                27
전환사채권 발행결정              22
자기주식 소각 결정              21
타법인 주식 및 출자증권 양도결정      20
소송 등의 제기                19
유형자산 양수 결정              11
회사분할 결정                  9
회사합병 결정                  9
품목허가 신청                  9
생산중단                     7
임상 계획 결과 발표              7
감자 결정                    4
자기주식취득 신탁계약 해지 결정        3
품목허가 승인                  3
임상 계획 철회                 3
기술이전계약체결                 3
영업양도 결정                  2
영업양수 결정                  2
품목허가 철회                  2
주식교환ㆍ이전 결정               1
Name: count, dtype: int64

In [6]:
df_after_telegram_filtered = df_after_telegram[df_after_telegram["disclosure_type"].isin(filtered_types)]
df_after_telegram_filtered

,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,report_name,summary_kr,period_dummy,disclosure_type
0,2022-07-01 01:03:17+00:00,-1.80,-0.66,0.14,0.19,-0.19,-0.19,0.28,-0.14,-0.14,...,-0.28,-0.66,-0.33,-0.80,-1.27,1,타법인주식및출자증권취득결정(자율공시),"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...",1,타법인 주식 및 출자증권 양수결정
1,2022-07-01 01:24:45+00:00,-1.57,-1.43,-1.43,-1.43,-1.28,-1.23,-1.19,-1.66,-1.66,...,0.48,0.10,0.48,0.10,0.53,1,단일판매ㆍ공급계약체결,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,1,단일판매ㆍ공급계약체결
2,2022-07-01 01:27:02+00:00,-0.05,0.10,0.00,0.05,-0.14,-0.14,-0.14,-0.05,0.14,...,-0.24,-0.24,-0.14,0.14,0.29,1,단일판매ㆍ공급계약체결(자회사의 주요경영사항),비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,1,단일판매ㆍ공급계약체결
3,2022-07-01 01:43:15+00:00,-0.89,-0.99,-1.11,-1.13,-1.03,-1.13,-1.18,-1.26,-1.34,...,0.74,0.09,0.75,-0.04,0.08,1,신규시설투자등(자율공시),동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,1,신규시설투자
4,2022-07-01 02:30:09+00:00,-1.91,-2.23,-1.81,-2.09,-1.91,-1.86,-1.91,-1.96,-1.78,...,-0.40,-0.67,-0.22,-0.40,-0.40,-1,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,1,임상 계획 신청
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,-4.21,-4.17,-4.18,-4.15,-3.62,-3.77,-3.76,-3.74,-3.80,...,-2.22,-3.14,-2.50,-2.73,-1.76,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...,1,단일판매ㆍ공급계약체결
8754,2023-12-28 04:54:23+00:00,-0.40,-0.40,-0.44,-0.47,-0.47,-0.11,-0.33,0.07,0.00,...,0.11,0.07,0.07,0.07,-0.14,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...,1,임상 계획 승인
8756,2023-12-28 05:42:24+00:00,-0.49,-0.50,-0.55,-0.59,-0.62,-0.62,-0.64,-0.64,-0.64,...,-1.22,-1.23,-0.91,-0.93,-0.86,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...,1,단일판매ㆍ공급계약체결
8757,2023-12-28 05:47:03+00:00,-5.07,-4.98,-4.98,-4.98,-4.95,-4.95,-5.07,-5.11,-5.11,...,4.48,3.83,4.34,3.92,3.73,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...,1,단일판매ㆍ공급계약체결


In [7]:
df_total = pd.concat([df_before_telegram, df_after_telegram_filtered])



In [8]:
df_total

,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,period_dummy,report_name,summary_kr,disclosure_type
0,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,-0.23,-0.24,-0.67,-0.34,-0.45,0,0,NaN,NaN,NaN
1,2021-01-04 10:08:00+00:00,NaN,NaN,-1.45,-1.64,-1.44,-2.24,-2.49,-2.69,-1.96,...,-1.24,-1.62,-1.37,-1.04,-1.91,1,0,NaN,NaN,NaN
2,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,-0.23,-0.24,-0.67,-0.34,-0.45,0,0,NaN,NaN,NaN
3,2021-01-04 10:14:00+00:00,-8.40,-8.59,-8.32,-8.32,-8.19,-8.65,-8.42,-8.36,-8.10,...,-3.51,-3.64,-4.17,-4.70,-4.90,1,0,NaN,NaN,NaN
4,2021-01-04 10:16:00+00:00,0.22,0.50,0.23,0.21,0.23,0.23,0.15,0.01,-0.31,...,-0.24,-0.06,-0.24,-0.50,0.03,-1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,-4.21,-4.17,-4.18,-4.15,-3.62,-3.77,-3.76,-3.74,-3.80,...,-2.22,-3.14,-2.50,-2.73,-1.76,1,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...,단일판매ㆍ공급계약체결
8754,2023-12-28 04:54:23+00:00,-0.40,-0.40,-0.44,-0.47,-0.47,-0.11,-0.33,0.07,0.00,...,0.11,0.07,0.07,0.07,-0.14,1,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...,임상 계획 승인
8756,2023-12-28 05:42:24+00:00,-0.49,-0.50,-0.55,-0.59,-0.62,-0.62,-0.64,-0.64,-0.64,...,-1.22,-1.23,-0.91,-0.93,-0.86,1,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...,단일판매ㆍ공급계약체결
8757,2023-12-28 05:47:03+00:00,-5.07,-4.98,-4.98,-4.98,-4.95,-4.95,-5.07,-5.11,-5.11,...,4.48,3.83,4.34,3.92,3.73,1,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...,단일판매ㆍ공급계약체결


# 회귀분석

### 📘 모형식

$$
\left| CAR_{post,i} - CAR_{pre,i} \right| = \alpha + \beta \cdot Telegram_i + \epsilon_i
$$

---

### 📊 변수 설명

**종속변수 (Dependent Variable)**  
이벤트 전후 CAR 변화폭의 절댓값:

$$
\left| CAR_{post,i} - CAR_{pre,i} \right|
$$


**설명변수 (Key Independent Variable)**  
텔레그램 도입 여부 변수:

$$
Telegram_i =
\begin{cases}
1, & \text{텔레그램 도입 이후 (After introduction)} \\
0, & \text{도입 이전 (Before introduction)}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{텔레그램 도입 이후와 이전 간 CAR 변화폭의 차이가 없다.}) \\
H_1 &: \beta < 0 \quad &(\text{텔레그램 도입 이후 CAR 변화폭이 감소했다.})
\end{aligned}
$$
---

### 📈 해석

이 회귀모형은 **텔레그램 도입 여부**가 이벤트 전후 누적초과수익률(CAR) 변화폭의 크기에 미치는 영향을 분석하기 위한 것입니다.  
β < 0 이고 통계적으로 유의하다면 귀무가설이 기각되어 공시 정보가 도입 이후 시장에 보다 신속하게 반영되어 정보 비효율성이 감소했음을 시사하며, 텔레그램 공시 알림 서비스가 시장 효율성 향상에 실질적으로 기여했음을 보여줍니다.


In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# df_total이 이미 존재한다고 가정
horizons = list(range(1, 11))
rows = []

for h in horizons:
    post_col = f"abn_ret_{h}m"
    pre_col  = f"abn_ret_minus_{h}m"
    if post_col not in df_total.columns or pre_col not in df_total.columns:
        raise KeyError(f"'{post_col}' 또는 '{pre_col}' 컬럼을 찾을 수 없습니다.")

    y = (df_total[post_col] - df_total[pre_col]).abs()
    X = sm.add_constant(df_total["period_dummy"], has_constant="add")

    valid = ~(y.isna() | X.isna().any(axis=1))
    y_ = y[valid]
    X_ = X[valid]

    model = sm.OLS(y_, X_).fit(cov_type="HC1")

    beta = model.params.get("period_dummy", np.nan)
    se   = model.bse.get("period_dummy", np.nan)
    tval = model.tvalues.get("period_dummy", np.nan)
    pval = model.pvalues.get("period_dummy", np.nan)

    rows.append({
        "horizon_min": h,
        "beta": beta,
        "std_err": se,
        "t_value": tval,
        "p_value": pval,
        "sig_0.05": "*" if (not pd.isna(pval) and pval < 0.05) else "",
        "n_obs": int(model.nobs),
        "r_squared": model.rsquared
    })

result_table = pd.DataFrame(rows).sort_values("horizon_min").reset_index(drop=True)

# ✅ p-value 포맷 조정 (지수표기 없이 소수점 8자리, 0은 "<1e-8"로 표시)
result_table["p_value"] = result_table["p_value"].apply(
    lambda x: f"{x:.12f}" if (x > 0 and not pd.isna(x)) else "<1e-8"
)

# 보기 좋게 출력
display_cols = ["horizon_min", "beta", "std_err", "t_value", "p_value", "sig_0.05", "n_obs", "r_squared"]
print(result_table[display_cols].to_string(index=False))

 horizon_min     beta  std_err  t_value        p_value sig_0.05  n_obs  r_squared
           1 0.180129 0.049490 3.639712 0.000272943633        *   7412   0.002035
           2 0.190550 0.056018 3.401571 0.000669996470        *   7377   0.001837
           3 0.187315 0.057088 3.281171 0.001033770015        *   7332   0.001724
           4 0.157618 0.055797 2.824834 0.004730516620        *   7287   0.001260
           5 0.142791 0.054938 2.599135 0.009345913048        *   7247   0.001079
           6 0.130914 0.054224 2.414316 0.015764788215        *   7205   0.000911
           7 0.123248 0.054427 2.264473 0.023545033078        *   7159   0.000809
           8 0.099680 0.054254 1.837263 0.066171077657            7108   0.000527
           9 0.108248 0.054250 1.995368 0.046002799632        *   7060   0.000629
          10 0.092887 0.055823 1.663954 0.096121581573            6792   0.000453


# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


# 중립 포함

In [10]:
import sys
sys.path.append('..')
from hypothesis_test import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral,
    logistic_hit_postCAR_with_neutral,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함) ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_total_sample = print_sample_summary_with_neutral(df, "label_sign")

# ΔCAR 기준 (pre/post window 차이 사용)
windows = list(range(1, 11))
results_delta_with_neutral = run_logistic_table(
    logistic_hit_delta_with_neutral, df_total_sample, windows, label="ΔCAR"
)

# post CAR 기준 (공시 이후만 사용)
results_post_with_neutral = run_logistic_table(
    logistic_hit_postCAR_with_neutral, df_total_sample, windows, label="post CAR")


=== Sample summary (including neutral) ===
Total events:        7,450
Neutrals:            2,087
  - Positive (1):    4,504
  - Negative (-1):   859


--- ΔCAR 기준 결과 ---
 t beta_star    std  t_stat      p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon
 1 0.2256*** 0.0541   4.174 2.995238e-05       1.253     0.661    0.710     4.86   7450 0.0019  0.0015         0.509880
 2 0.2889*** 0.0549   5.263 1.415390e-07       1.335     0.668    0.728     6.08   7450 0.0030  0.0026         0.570988
 3 0.2837*** 0.0539   5.265 1.399550e-07       1.328     0.648    0.709     6.17   7450 0.0030  0.0025         0.612063
 4 0.2668*** 0.0535   4.984 6.244410e-07       1.306     0.643    0.701     5.87   7450 0.0026  0.0022         0.627468
 5 0.2941*** 0.0534   5.511 3.570400e-08       1.342     0.634    0.699     6.52   7450 0.0032  0.0028         0.621327
 6 0.2981*** 0.0531   5.617 1.948000e-08       1.347     0.626    0.693     6.68   7450 0.0033  0.0029        

# 중립 제거

In [12]:
import sys
sys.path.append('..')
from hypothesis_test import (
    print_sample_summary,
    logistic_hit_delta,
    logistic_hit_postCAR,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")

# ΔCAR 기준 (pre/post window 차이 사용)
windows = list(range(1, 11))
results_delta = run_logistic_table(logistic_hit_delta, df_nn, windows, label="ΔCAR", include_r2=False)

# post CAR 기준 (공시 이후만 사용)
results_post = run_logistic_table(logistic_hit_postCAR, df_nn, windows, label="post CAR", include_r2=False)


=== Sample summary (neutral removed) ===
Total events:        7,450
Removed neutrals:    2,087
Used (non-neutral):  5,363
  - Positive (1):    4,504
  - Negative (-1):   859


--- ΔCAR 기준 결과 ---
 t  beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs
 1 -0.3010*** 0.0634  -4.749 0.000002       0.740     0.772    0.715    -5.72   5363
 2  -0.1889** 0.0641  -2.946 0.003221       0.828     0.771    0.735    -3.51   5363
 3   -0.1434* 0.0625  -2.294 0.021784       0.866     0.747    0.719    -2.81   5363
 4    -0.1187 0.0618  -1.922 0.054576       0.888     0.734    0.710    -2.38   5363
 5    -0.0517 0.0613  -0.843 0.399320       0.950     0.720    0.710    -1.05   5363
 6    -0.0460 0.0608  -0.757 0.449175       0.955     0.712    0.702    -0.95   5363
 7    -0.0688 0.0600  -1.147 0.251572       0.934     0.700    0.686    -1.46   5363
 8   -0.1241* 0.0593  -2.094 0.036289       0.883     0.693    0.666    -2.70   5363
 9    -0.0845 0.0593  -1.424 0.154341   

정보의 선반영(anticipation)·사전확산 강화
- 텔레그램 도입 이후, 공시 직전에 감성 방향으로 미리 움직이는 비율/크기가 커졌을 가능성
- 그래서 공시 순간의 post CAR 방향 정합은 높아지지만(β_post>0), “직전 대비 추가로 얼마나 같은 방향으로 더 가느냐”를 보는 ΔCAR 기준에서는 추가분이 줄어 **정합성↓**로 관측(β_Δ<0)

과잉반응 감소 / 즉시 반영 후 빠른 수렴
- 도입 전에는 공시 직후 같은 방향으로 과도하게 더 밀던 패턴이, 도입 후에는 이미 반영되어 추가 밀림이 줄어듦 → Δ 기준 정합성 감소(β_Δ<0).
- 하지만 “초기 방향” 자체는 더 정확해짐 → post 기준 정합성 증가(β_post>0).


도입 후 시장은 감성 방향으로 “빠르게 맞게 움직이지만”, 직전에 이미 반영된 탓에 공시 순간의 “추가” 같은 방향 밀림은 줄어든다. 이는 “정보의 더 빠른 확산·반영”이라는 효율성 개선 스토리와 일관됩니다.